# Treinamento de Modelo YOLO Customizado (60 épocas)

Este notebook implementa o treinamento de um modelo YOLO customizado para detectar duas categorias de objetos visualmente distintas. Utilizaremos o YOLOv5 com 60 épocas de treinamento para comparar com o modelo de 30 épocas.

## 1. Configuração do Ambiente

Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente usando o script setup_env.sh.

In [1]:
# Verificar se o ambiente já foi configurado
import os
import sys

# Se o ambiente ainda não foi configurado, execute o setup_env.sh
if not os.path.exists('../yolov5'):
    print("Configurando o ambiente com setup_env.sh...")
    !chmod +x ../setup_env.sh
    !../setup_env.sh
else:
    print("Ambiente já configurado.")

# Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import yaml
from pathlib import Path
from tqdm.notebook import tqdm

# Adicionar o diretório YOLOv5 ao path
yolov5_path = os.path.abspath('../yolov5')
if yolov5_path not in sys.path:
    sys.path.append(yolov5_path)
    print(f"Adicionado {yolov5_path} ao sys.path")

Ambiente já configurado.
Adicionado /Users/gab/Documents/CodePlay/@fiap/fase6_cap1/yolov5 ao sys.path


## 2. Verificação do Dataset e Configuração

Vamos verificar se o dataset está organizado corretamente e se o arquivo de configuração YAML existe.

In [2]:
# Verificar se o dataset está pronto
dataset_dirs = ['../dataset/train/images', '../dataset/train/labels', 
                '../dataset/val/images', '../dataset/val/labels', 
                '../dataset/test/images', '../dataset/test/labels']

for dir_path in dataset_dirs:
    if not os.path.exists(dir_path):
        print(f"❌ Diretório não encontrado: {dir_path}")
    else:
        print(f"✅ Diretório encontrado: {dir_path}")

# Verificar se o arquivo data.yaml existe
if os.path.exists('../data/data.yaml'):
    # Carregar o arquivo
    with open('../data/data.yaml', 'r') as f:
        data_yaml_content = yaml.safe_load(f)
    
    print("\nArquivo data.yaml encontrado!")
    print("Conteúdo:")
    print(yaml.dump(data_yaml_content, sort_keys=False))
    
    # Extrair as categorias
    categorias = data_yaml_content['names']
else:
    # Se o arquivo não existir, criar um novo
    print("\n❌ Arquivo data.yaml não encontrado.")
    categorias = ['apple', 'banana']  # Nossas categorias são maçãs e bananas
    
    # Obter o caminho absoluto para o diretório do dataset
    dataset_dir = os.path.abspath('../dataset')
    
    # Criar o arquivo data.yaml com a estrutura correta para o YOLOv5
    data_yaml = {
        'train': os.path.join(dataset_dir, 'train', 'images'),
        'val': os.path.join(dataset_dir, 'val', 'images'),
        'test': os.path.join(dataset_dir, 'test', 'images'),
        'nc': len(categorias),
        'names': categorias
    }
    
    # Salvar o arquivo
    with open('../data/data.yaml', 'w') as f:
        yaml.dump(data_yaml, f, sort_keys=False)
    
    print("Arquivo data.yaml criado com sucesso!")
    print("Conteúdo:")
    print(yaml.dump(data_yaml, sort_keys=False))

✅ Diretório encontrado: ../dataset/train/images
✅ Diretório encontrado: ../dataset/train/labels
✅ Diretório encontrado: ../dataset/val/images
✅ Diretório encontrado: ../dataset/val/labels
✅ Diretório encontrado: ../dataset/test/images
✅ Diretório encontrado: ../dataset/test/labels

Arquivo data.yaml encontrado!
Conteúdo:
path: ../dataset
train: train/images
val: val/images
test: test/images
nc: 2
names:
- apple
- banana



## 3. Treinamento do Modelo YOLO (60 épocas)

Vamos verificar se o modelo já foi treinado. Se sim, usaremos os resultados existentes. Caso contrário, treinaremos o modelo.

In [3]:
# Definir parâmetros de treinamento
epochs = 60  # Dobro do número de épocas do modelo anterior
batch_size = 16
img_size = 640
model_type = 'yolov5s'  # Mesmo modelo para comparação justa
model_dir = '../models/yolo_custom_60epochs'
weights_path = os.path.join(model_dir, 'weights/best.pt')
results_file = os.path.join(model_dir, 'results.csv')

# Verificar se o modelo já foi treinado
if os.path.exists(weights_path) and os.path.exists(results_file):
    print(f"\n✅ Modelo já treinado encontrado em {model_dir}")
    print(f"✅ Arquivo de pesos encontrado em {weights_path}")
    print(f"✅ Arquivo de resultados encontrado em {results_file}")
    print("\nUsando modelo existente para análise e validação.")
else:
    print(f"\n❌ Modelo treinado não encontrado ou incompleto.")
    print("Iniciando treinamento do modelo...")
    
    # Obter o caminho absoluto para o arquivo data.yaml
    data_yaml_path = os.path.abspath('../data/data.yaml')
    
    # Comando de treinamento
    !cd ../yolov5 && python train.py \
        --img {img_size} \
        --batch {batch_size} \
        --epochs {epochs} \
        --data {data_yaml_path} \
        --weights {model_type}.pt \
        --project ../models \
        --name yolo_custom_60epochs \
        --cache


❌ Modelo treinado não encontrado ou incompleto.
Iniciando treinamento do modelo...
train: weights=yolov5s.pt, cfg=, data=/Users/gab/Documents/CodePlay/@fiap/fase6_cap1/data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=60, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../models, name=yolo_custom_60epochs, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-416-gfe1d4d99 Python-3.12.6 torch-2.7.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937

## 4. Análise dos Resultados do Treinamento

Vamos analisar os resultados do treinamento, incluindo as métricas de desempenho e as curvas de aprendizado.

In [ ]:
# Carregar os resultados do treinamento
results_file = '../models/yolo_custom_60epochs/results.csv'

if os.path.exists(results_file):
    try:
        results = pd.read_csv(results_file)
        
        # Plotar as curvas de aprendizado
        plt.figure(figsize=(15, 10))
        
        # Plotar perda de treinamento
        plt.subplot(2, 2, 1)
        if '      train/box_loss' in results.columns:
            plt.plot(results['               epoch'], results['      train/box_loss'], label='train/box_loss')
        if '      train/obj_loss' in results.columns:
            plt.plot(results['               epoch'], results['      train/obj_loss'], label='train/obj_loss')
        if '      train/cls_loss' in results.columns:
            plt.plot(results['               epoch'], results['      train/cls_loss'], label='train/cls_loss')
        plt.xlabel('Época')
        plt.ylabel('Perda')
        plt.title('Perdas de Treinamento')
        plt.legend()
        plt.grid(True)
        
        # Plotar perda de validação
        plt.subplot(2, 2, 2)
        if '        val/box_loss' in results.columns:
            plt.plot(results['               epoch'], results['        val/box_loss'], label='val/box_loss')
        if '        val/obj_loss' in results.columns:
            plt.plot(results['               epoch'], results['        val/obj_loss'], label='val/obj_loss')
        if '        val/cls_loss' in results.columns:
            plt.plot(results['               epoch'], results['        val/cls_loss'], label='val/cls_loss')
        plt.xlabel('Época')
        plt.ylabel('Perda')
        plt.title('Perdas de Validação')
        plt.legend()
        plt.grid(True)
        
        # Verificar quais métricas estão disponíveis
        available_metrics = [col for col in results.columns if 'metrics/' in col]
        print(f"Métricas disponíveis: {available_metrics}")
        
        # Plotar métricas de precisão se disponíveis
        if available_metrics:
            plt.subplot(2, 2, 3)
            for metric in available_metrics:
                plt.plot(results['               epoch'], results[metric], label=metric.replace('metrics/', ''))
            plt.xlabel('Época')
            plt.ylabel('Valor')
            plt.title('Métricas de Desempenho')
            plt.legend()
            plt.grid(True)
        
        # Plotar tempo de treinamento se disponível
        time_columns = [col for col in results.columns if 'time' in col.lower()]
        if time_columns:
            plt.subplot(2, 2, 4)
            for col in time_columns:
                plt.plot(results['               epoch'], results[col], label=col.strip())
            plt.xlabel('Época')
            plt.ylabel('Tempo (s)')
            plt.title('Tempo por Época')
            plt.legend()
            plt.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # Mostrar as métricas finais
        final_results = results.iloc[-1]
        print("\nMétricas finais após 60 épocas:")
        
        # Mostrar perdas
        if '      train/box_loss' in results.columns:
            print(f"Box Loss (Treino): {final_results['      train/box_loss']:.4f}")
        if '      train/obj_loss' in results.columns:
            print(f"Object Loss (Treino): {final_results['      train/obj_loss']:.4f}")
        if '      train/cls_loss' in results.columns:
            print(f"Class Loss (Treino): {final_results['      train/cls_loss']:.4f}")
        if '        val/box_loss' in results.columns:
            print(f"Box Loss (Val): {final_results['        val/box_loss']:.4f}")
        if '        val/obj_loss' in results.columns:
            print(f"Object Loss (Val): {final_results['        val/obj_loss']:.4f}")
        if '        val/cls_loss' in results.columns:
            print(f"Class Loss (Val): {final_results['        val/cls_loss']:.4f}")
        
        # Mostrar métricas de desempenho disponíveis
        for metric in available_metrics:
            metric_name = metric.replace('metrics/', '')
            print(f"{metric_name}: {final_results[metric]:.4f}")
        
        # Mostrar tempo de treinamento se disponível
        if '      total_epochs' in results.columns and time_columns:
            time_col = time_columns[0]  # Usar a primeira coluna de tempo disponível
            print(f"Tempo total de treinamento: {final_results['      total_epochs'] * final_results[time_col]:.2f} segundos")
    except Exception as e:
        print(f"Erro ao analisar os resultados: {e}")
        print("Tentando mostrar as colunas disponíveis no arquivo de resultados:")
        try:
            results = pd.read_csv(results_file)
            print(f"Colunas disponíveis: {results.columns.tolist()}")
        except Exception as e2:
            print(f"Erro ao ler o arquivo de resultados: {e2}")
else:
    print("Arquivo de resultados não encontrado. Verifique se o treinamento foi concluído com sucesso.")

## 5. Visualização de Algumas Predições

Vamos visualizar algumas predições do modelo treinado em imagens do conjunto de validação.

In [ ]:
# Usar o script detect.py do YOLOv5 para fazer predições
model_path = '../models/yolo_custom_60epochs/weights/best.pt'

if os.path.exists(model_path):
    try:
        # Usar o script detect.py do YOLOv5 para fazer predições
        val_img_dir = '../dataset/val/images'
        output_dir = '../models/yolo_custom_60epochs/predictions'
        os.makedirs(output_dir, exist_ok=True)
        
        # Executar o script detect.py
        !cd ../yolov5 && python detect.py \
            --weights {model_path} \
            --source {val_img_dir} \
            --conf 0.25 \
            --project {os.path.dirname(output_dir)} \
            --name {os.path.basename(output_dir)} \
            --save-txt \
            --save-conf
        
        # Mostrar algumas imagens com predições
        pred_img_dir = output_dir
        pred_img_files = [os.path.join(pred_img_dir, f) for f in os.listdir(pred_img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        # Selecionar algumas imagens aleatórias
        import random
        random.seed(42)  # Para reprodutibilidade
        sample_imgs = random.sample(pred_img_files, min(5, len(pred_img_files)))
        
        # Mostrar as imagens
        plt.figure(figsize=(15, 12))
        for i, img_path in enumerate(sample_imgs):
            try:
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                plt.subplot(len(sample_imgs), 1, i+1)
                plt.imshow(img)
                plt.title(f"Detecções em {os.path.basename(img_path)}")
                plt.axis('off')
            except Exception as e:
                print(f"Erro ao processar {img_path}: {e}")
        
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Erro ao executar o script detect.py: {e}")
        import traceback
        traceback.print_exc()
else:
    print("Modelo treinado não encontrado. Verifique se o treinamento foi concluído com sucesso.")

## 6. Conclusão

Neste notebook, treinamos (ou usamos um modelo já treinado) YOLO customizado com 60 épocas para detectar duas categorias de objetos. Analisamos as métricas de desempenho e visualizamos algumas predições.

No próximo notebook, vamos validar e testar ambos os modelos (30 e 60 épocas) e fazer uma comparação detalhada entre eles.